# Imports

In [ ]:
import mdtraj as md
import numpy as np
from datetime import datetime
from tqdm import tqdm
from pathlib import Path
import yaml

## Paths

In [ ]:
idir = Path("/Users/alexpayne/lilac-mount-point/asap-datasets/current/sars_01_prepped_v3")
pdb_fn = Path("/Users/alexpayne/lilac-mount-point/asap-datasets/current/sars_01_prepped_v3/Mpro-x0072_0A_AAR-POS-d2a4d1df-1/Mpro-x0072_0A_AAR-POS-d2a4d1df-1_prepped_receptor_0.pdb")
pdb_fn2 = Path("/Users/alexpayne/lilac-mount-point/asap-datasets/current/sars_01_prepped_v3/Mpro-P0010_0A_PET-UNK-c9c1e0d8-4/Mpro-P0010_0A_PET-UNK-c9c1e0d8-4_prepped_receptor_0.pdb")
sars_selection_fn = Path("../../../metadata/sars2.yaml")

In [ ]:
pdb_fn.exists()

In [ ]:
pdb_fn2.exists()

## load files

In [ ]:
pdb1 = md.load(pdb_fn)

In [ ]:
pdb2 = md.load(pdb_fn2)

In [ ]:
with open(sars_selection_fn, 'r') as f:
    sars_sel = yaml.safe_load(f)

In [ ]:
sars_sel

In [ ]:
def pymol_to_mdtraj(pymol_string):
    """
    Simple function to convert a pymol-like residue string into mdtraj-like
    """
    x = pymol_string.replace('+', ' or residue ')
    x = x.replace('-', ' to ')
    x = x.replace('resi ', 'residue ')
    x = x.replace("polymer.protein", "")
    return x

In [ ]:
for selection in sars_sel.values():
    print(pymol_to_mdtraj(selection))

# Examine topology

In [ ]:
pdb1.topology

In [ ]:
pdb2.topology

In [ ]:
pdb1.atom_slice(pdb1.topology.select("chainid 0"))

In [ ]:
pdb2.topology.chain(1).topology

In [ ]:
def make_selection(t, string_):
    print(t.atom_slice(t.topology.select(string_)))

In [ ]:
string_ = "chainid 0"
for pdb in [pdb1, pdb2]:
    make_selection(pdb, string_)

In [ ]:
string_ = "chainid 1"
for pdb in [pdb1, pdb2]:
    make_selection(pdb, string_)

In [ ]:
string_ = "chainid 2"
for pdb in [pdb1, pdb2]:
    make_selection(pdb, string_)

In [ ]:
string_ = "chainid 3"
for pdb in [pdb1, pdb2]:
    make_selection(pdb, string_)

In [ ]:
string_ = "(chainid 0 or chainid 1) and resid 41"
for pdb in [pdb1, pdb2]:
    make_selection(pdb, string_)

In [ ]:
string_ = "not (name =~ 'H.*') and chainid 2"
for pdb in [pdb1, pdb2]:
    make_selection(pdb, string_)

## doin it

In [ ]:
full_protein = "not element H and (chainid 0 or chainid 2)"
active_site_string = "not element H and (chainid 0 or chainid 2) and (residue 140 to 145 or residue 163 or residue 172 or residue 25 to 27 or residue 41 or residue 49 or residue 54 or residue 165 to 168 or residue 189 to 192)"

In [ ]:
for pdb in [pdb1, pdb2]:
    make_selection(pdb, full_protein)

In [ ]:
md.rmsd(pdb1, pdb2, atom_indices=pdb1.topology.select(active_site_string))

In [ ]:
md.rmsd(pdb1, pdb2, atom_indices=pdb2.topology.select(active_site_string))

In [ ]:
pdb1_active_site = pdb1.atom_slice(pdb1.topology.select(active_site_string))

In [ ]:
pdb2_active_site = pdb2.atom_slice(pdb2.topology.select(active_site_string))

In [ ]:
pdb2_active_site

In [ ]:
pdb1_active_site

In [ ]:
t = md.join([pdb1_active_site, pdb2_active_site])

In [ ]:
t.save_pdb("combined_test.pdb")

In [ ]:
help(t.save_pdb)

In [ ]:
md.shrake_rupley(t)

# Iterate through all PDBs 

In [ ]:
import logging
def analyze_mp(fn, out_dir):
    full_protein_selection = "not element H and (chainid 0 or chainid 2)"
    active_site_string = "not element H and (chainid 0 or chainid 2) and (residue 140 to 145 or residue 163 or residue 172 or residue 25 to 27 or residue 41 or residue 49 or residue 54 or residue 165 to 168 or residue 189 to 192)"
    output_name = fn.stem

    ## Prepare logger
    handler = logging.FileHandler(out_dir / f"{output_name}-log.txt", mode="w")
    prep_logger = logging.getLogger(output_name)
    prep_logger.setLevel(logging.INFO)
    prep_logger.addHandler(handler)
    prep_logger.info(datetime.isoformat(datetime.now()))

    prep_logger.info(f"Loading {fn}")
    pdb = md.load(fn)

    active_site_idx = pdb.topology.select(active_site_string)
    active_site = pdb.atom_slice(active_site_idx)

    full_protein_idx = pdb.topology.select(full_protein_selection)
    full_protein = pdb.atom_slice(full_protein_idx)

    prep_logger.info(f"Saving idx arrays to {out_dir}")
    np.save(out_dir / f"{output_name}_active_site.npy", active_site_idx)
    np.save(out_dir / f"{output_name}_full_protein.npy", full_protein_idx)

    prep_logger.info(f"Saving pdbs to {out_dir}")
    active_site.save(out_dir / f"{output_name}_active_site.pdb")
    full_protein.save(out_dir / f"{output_name}_full_protein.pdb")

    return True

In [ ]:
analyze_mp(pdb_fn, Path("."))

In [ ]:
pdb_fn.stem

## Try loading all sliced

In [ ]:
idir = Path("/Users/alexpayne/lilac-mount-point/asap-datasets-ap/sars_01_prepped_v3_split/")

In [ ]:
pdbs = list(idir.glob("*active_site.pdb"))

In [ ]:
len(pdbs)

In [ ]:
ts = [md.load(pdb) for pdb in tqdm(pdbs)]

In [ ]:
ids = np.array(list([pdbs[i] for i, t in enumerate(ts)]))

In [ ]:
lens = np.array(list([t.n_atoms for i, t in enumerate(ts)]))

In [ ]:
ids[lens == 354]

In [ ]:
sum(lens == 354)

In [ ]:
ids[lens != 354]

In [ ]:
same_atoms = ids[lens == 354]

In [ ]:
combined = md.load(list(same_atoms))

## calculate RMSD

In [ ]:
rmsd_array = []
for i in range(combined.n_frames):
    rmsd_array.append(md.rmsd(combined, combined, frame=i))
rmsd_matrix = np.matrix(rmsd_array)

In [ ]:
np.save("527_rmsd_array.npy", rmsd_matrix)

In [ ]:
np.shape(rmsd_array)

In [ ]:
rmsd_array * 10

In [ ]:
np.shape(rmsd_matrix)

In [ ]:
rmsd_matrix * 10

In [ ]:
combined.save_pdb("527_combined.pdb")

# load combined pdb

In [ ]:
combined = md.load("527_combined.pdb")

# Cluster

In [ ]:
from sklearn.decomposition import PCA
import numpy as np

In [ ]:
rmsd_matrix = np.load("527_rmsd_array.npy")

## calculate PCA

In [ ]:
pca = PCA(n_components='mle')

In [ ]:
pca.fit(rmsd_matrix)

In [ ]:
pca.n_components

In [ ]:
pca.explained_variance_

## plot with matplotlib

In [ ]:
import matplotlib.pyplot as plt
cum_sum_exp = np.cumsum(pca.explained_variance_)
plt.bar(range(0,len(pca.explained_variance_)), pca.explained_variance_, alpha=0.5, align='center', label='Individual explained variance')
plt.step(range(0,len(cum_sum_exp)), cum_sum_exp, where='mid',label='Cumulative explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal component index')
plt.legend(loc='best')
plt.tight_layout()
plt.show()

In [ ]:
pca.n_components_

In [ ]:
transformed = pca.transform(rmsd_matrix)

In [ ]:
np.shape(transformed)

In [ ]:
np.shape(transformed[:,0:2])

In [ ]:
plt.scatter(transformed[:,0], transformed[:,1])

In [ ]:
plt.scatter(transformed[:,0], transformed[:,2])

In [ ]:
plt.scatter(transformed[:,1], transformed[:,2])

## kmeans cluster

In [ ]:
plt.scatter(transformed[:,2], transformed[:,0])

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(n_clusters=10)

In [ ]:
kmeans.fit(rmsd_matrix)

In [ ]:
kmeans.labels_

## plot with labels

In [ ]:
plt.scatter(transformed[:,0], transformed[:,1], c=kmeans.labels_)

In [ ]:
plt.scatter(transformed[:,0], transformed[:,2], c=kmeans.labels_)

In [ ]:
plt.scatter(transformed[:,1], transformed[:,2], c=kmeans.labels_)

In [ ]:
x = 5
y = 5
fig, axs = plt.subplots(x,y)
for i in range(x):
    for j in range(y):
        ax = axs[i,j]
        ax.scatter(transformed[:,i], transformed[:,j], c=kmeans.labels_)

In [ ]:
fig

## convert to pandas array

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(transformed[:, 0:5])

In [ ]:
df["Cluster"] = kmeans.labels_

In [ ]:
df

In [ ]:
pd.plotting.scatter_matrix(df)

In [ ]:
import plotly.express as px

In [ ]:
fig = px.scatter_matrix(df, dimensions=[0,1,2,3,4], color='Cluster')

In [ ]:
fig.show()